# Ch06. 토픽 모델링(Topic Modeling)

# v02. 잠재 디리클레 할당(Latent Dirichlet Allocation, LDA)

**토픽 모델링**

- 문서의 집합에서 토픽을 찾아내는 프로세스
- 검색 엔진, 고객 민원 시스템 등과 같이 문서의 주제를 알아내는 일이 중요한 곳에서 사용됨
- LDA은 토픽 모델링의 대표적인 알고리즘이다.

**LDA**

- 문서들은 토픽들의 혼합으로 구성되어져 있으며, 토픽들은 확률 분포에 기반하여 단어들을 생성한다고 가정한다.
- 데이터가 주어지면, LDA는 문서가 생성되던 과정을 역추적한다.
- [참고 링크](https://lettier.com/projects/lda-topic-modeling/)
  - 코드 작성 없이 입력한 문서들로부터 DTM을 만들고 LDA를 수행한 결과를 보여주는 웹사이트

<br>

## 2.1 잠재 디리클레 할당 (Latent Dirichlet Allocation, LDA) 개요

### 2.1.1 LDA 블랙박스

- LDA에 문서 집합을 입력하면, 어떤 결과를 보여주는 지 간소화된 예 확인  

**1) 3개의 문서**

- 아래와 같은 3개의 문서가 있다고 하자.
- 지금의 예제는 간단해서 눈으로도 토픽 모델링을 할 수 있을 것 같지만, 실제 수십만개 이상의 문서가 있는 경우는 직접 토픽을 찾아내는 것이 어렵기 때문에 LDA의 도움이 필요하다.

> 문서1 : 저는 사과랑 바나나를 먹어요  
문서2 : 우리는 귀여운 강아지가 좋아요  
문서3 : 저의 깜찍하고 귀여운 강아지가 바나나를 먹어요

**2) 토픽의 개수 지정**

- LDA를 수행할 때 문서 집합에서 토픽이 몇 개가 존재할 지 가정하는 것은 사용자가 해야 할 일이다.
- 여기서는 LDA에 2개의 토픽을 찾으라고 요청한다.
  - 토픽의 개수를 의미하는 변수를 k라고 했을 때, k를 2로 한다.
- k의 값을 잘못 선택하면 원치않는 이상한 결과가 나올 수 있다.
- 이렇게 모델의 성능에 영향을 주는 사용자가 직접 선택하는 매개변수를 **하이퍼파라미터**라고 한다.
- 하이퍼파라미터의 선택은 여러 실험을 통해 얻은 값일 수도 있고, 우선 시도해보는 값일 수도 있다.

<br>

**3) LDA 수행 결과**

- LDA가 위의 세 문서로부터 2개의 토픽을 찾은 결과는 아래와 같다.
- 여기서는 LDA 입력 전에 주어와 불필요한 조사 등을 제거하는 전처리 과정은 거쳤다고 가정한다.
  - 즉, 전처리 과정을 거친 DTM이 LDA의 입력이 되었다고 가정한다.

- LDA는 **각 문서의 토픽 분포**와 **각 토픽 내의 단어 분포**를 추정한다.

**3-1) 각 문서의 토픽 분포**

- 문서1
  - 토픽 A 100%
- 문서2
  - 토픽 B 100%
- 문서3
  - 토픽 B 60%
  - 토픽 A 40%

**3-2) 각 토픽의 단어 분포**

- 토픽 A
  - **사과 20%**
  - **바나나 40%**
  - **먹어요 40%**
  - 귀여운 0%
  - 강아지 0%
  - 깜찍하고 0%
  - 좋아요 0%
- 토픽 B
  - 사과 0%
  - 바나나 0%
  - 먹어요 0%
  - **귀여운 33%**
  - **강아지 33%**
  - **깜찍하고 16%**
  - **좋아요 16%**

<br>

**4) LDA 결과 해석**

- LDA는 토픽의 제목을 정해주지 않지만, 이 시점에서 알고리즘의 사용자는 위 결과로부터 두 토픽이 각각 과일에 대한 토픽과 강아지에 대한 토픽이라고 판단해볼 수 있다.

<br>

## 2.2 LDA의 가정

- LDA는 문서의 집합으로부터 어떤 토픽이 존재하는 지를 알아내기 위한 알고리즘이다.
- LDA는 빈도수 기반의 표현 방법인 BoW의 행렬 DTM 또는 TF-IDF 행렬을 입력으로 한다.  
$\rightarrow$ LDA는 단어의 순서는 신경쓰지 않겠다는 것을 알 수 있다.

<br>

### 2.2.1 LDA가 가정하는 문서 작성 시나리오

- LDA는 문서들로부터 토픽을 뽑아내기 위해서 이러한 가정을 염두해두고 있다.
  - 모든 문서 하나, 하나가 작성될 때 그 문서의 작성자는 다음과 같은 생각을 했을 것이다.
  > "나는 이 문서를 작성하기 위해서 이런 주제들을 넣을거고, 이런 주제들을 위해서는 이런 단어들을 넣을 거야"

- 각각의 문서는 다음과 같은 과정을 거쳐서 작성되었다고 가정한다.

**1) 문서에 사용할 단어의 개수 N을 정한다.**

- ex) 5개의 단어를 사용하기로 한다.

**2) 문서에 사용할 토픽의 혼합을 확률 분포에 기반하여 결정한다.**

- ex) 위 예제와 같이 토픽이 2개라고 했을 때 강아지 토픽을 60%, 과일 토픽을 40%와 같이 선택할 수 있다.

**3) 문서에 사용할 각 단어를 아래와 같이 정한다.**

**3-1) 토픽 분포에서 토픽 T를 확률적으로 고른다.**

- ex) 60% 확률로 강아지 토픽을 선택하고, 40%의 확률로 과일 토픽을 선택할 수 있다.

**3-2) 선택한 토픽 T에서 단어의 출현 확률 분포에 기반에 문서에 사용할 단어를 고른다.**

- ex) 강아지 토픽을 선택 $\rightarrow$ 33% 확률로 강아지란 단어를 선택할 수 있다.

<br>

- 이제 **3)**을 반복하면서 문서를 완성한다.

- 이러한 과정을 통해 문서가 작성되었다는 가정 하에 LDA는 토픽을 뽑아내기 위하여 위 과정을 역으로 추적하는 **역공학(reverse engneering)**을 수행한다.

<br>

## 2.3 LDA 수행하기

### 2.3.1 LDA의 수행 과정 정리

**1) 사용자는 알고리즘에게 토픽의 개수 k를 알려준다**

- LDA에게 토픽의 개수를 알려주는 역할은 사용자의 역할이다.
- LDA는 토픽의 개수 k를 입력받으면, k개의 토픽이 M개의 전체 문서에 걸쳐 분포되어 있다고 가정한다.

<br>

**2) 모든 단어를 k개 중 하나의 토픽에 할당한다.**

- LDA는 모든 문서의 모든 단어에 대해 k개 중 하나의 토픽을 랜덤으로 할당한다.
- 이 작업이 끝나면 각 문서는 토픽을 가지며, 토픽은 단어 분포를 가지는 상태이다.
- 랜덤으로 할당했기 때문에 이 결과는 전부 틀린 상태이다.
- 만약 한 단어가 한 문서에서 2회 이상 등장하였다면, 각 단어는 서로 다른 토픽에 할당되었을 수도 있다.

<br>

**3) 모든 문서의 모든 단어에 대해서 아래의 사항을 반복 진행한다. (iterative)**

**3-1) 어떤 문서의 각 단어 w는 자신은 잘못된 토픽에 할당되어져 있지만, 다른 단어들은 전부 올바른 토픽에 할당되어져 있는 상태라고 가정한다.**

- 이에 따라 단어 w는 아래의 두 가지 기준에 따라서 토픽이 재할당된다.

- $p(topic \, t \; | \; document \, d)$
  - 문서 $d$의 단어들 중 토픽 $t$에 해당하는 단어들의 비율
- $p(word \, w \; | \; topic \, t)$
  - 단어 $w$를 갖고 있는 모든 문서들 중 토픽 $t$가 할당된 비율

- 이를 반복하면, 모든 할당이 완료된 수렴 상태가 된다.
- 두 가지 기준이 어떤 의미인 지 확인
- 설명의 편의를 위해서 두 개의 문서라는 새로운 예를 사용한다.

<img src="https://wikidocs.net/images/page/30708/lda1.PNG" />

- 위의 그림은 두 개의 문서 doc1과 doc2를 보여준다.
- 여기서는 doc1의 세 번째 단어 apple의 토픽을 결정하고자 한다.

<img src="https://wikidocs.net/images/page/30708/lda3.PNG" />

- 우선 첫 번째로 사용하는 기준은 문서 doc1의 단어들이 어떤 토픽에 해당하는 지를 확인한다.
- doc1의 모든 단어들은 토픽 A와 토픽 B에 50 대 50의 비율로 할당되어져 있다.
- 이 기준에 따르면 단어 apple은 토픽 A 또는 토픽 B 둘 중 어디에도 속할 가능성이 있다.

<img src="https://wikidocs.net/images/page/30708/lda2.PNG" />

- 두 번째 기준은 단어 apple이 전체 문서에서 어떤 토픽에 할당되어져 있는 지를 본다.
- 이 기준에 따르면 단어 apple은 토픽 B에 할당된 가능성이 높다.


<br>

- 이러한 두 가지 기준을 참고하여 LDA는 doc1의 apple을 어떤 토픽에 할당할 지 결정한다.

<br>

## 2.4 잠재 디리클레 할당과 잠재 의미 분석의 차이

### 2.4.1 잠재 의미 분석 (Latent Semantic Analysis, LSA)

- DTM을 차원 축소하여 축소 차원에서 근접 든어들을 토픽으로 묶는다.

<br>

### 2.4.2 잠재 디리클레 할당 (Latent Dirichlet Allocation, LDA)

- 단어가 특정 토픽에 존재할 확률과 문서에 특정 토픽이 존재할 확률을 결합확률로 추정하여 토픽을 추출한다.

<br>

## 2.5 실습을 통한 이해

- 이번 챕터에서는 `gensim`을 사용한다.
- 사이킷런을 통해 LDA를 진행하는 실습은 [해당 링크](https://wikidocs.net/40710)를 참고하면 된다.

<br>

### 2.5.1 정수 인코딩과 단어 집합 만들기

- Twenty Newsgroups 이라고 불리는 20개의 다른 주제를 가진 뉴스 데이터를 다시 사용한다.
- 동일한 전처리 과정을 거친 후에 `tokenized_doc`으로 저장한 상태라고 하자.

In [12]:
import pandas as pd
from sklearn.datasets import fetch_20newsgroups

dataset = fetch_20newsgroups(shuffle=True,
                             random_state=1,
                             remove=('headers', 'fotters', 'quotes'))

documents = dataset.data

news_df = pd.DataFrame({'document':documents})

# 특수문자 제거
news_df['clean_doc'] = news_df['document'].str.replace("[^a-zA-Z]", " ")

# 길이가 3이하인 단어 제거 (길이가 짧은 단어 제거)
news_df['clean_doc'] = news_df['clean_doc'].apply(lambda x: ' '.join([w for w in x.split() if len(w)>3]))


# 전체 단어에 대한 소문자 변환
news_df['clean_doc'] = news_df['clean_doc'].apply(lambda x: x.lower())


import nltk
nltk.download('stopwords')

from nltk.corpus import stopwords

stop_words = stopwords.words('english')

# 토큰화
tokenized_doc = news_df['clean_doc'].apply(lambda x: x.split()) 

# 불용어 제거
tokenized_doc = tokenized_doc.apply(lambda x: [item for item in x if item not in stop_words])

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [13]:
tokenized_doc[:5]

0    [well, sure, story, seem, biased, disagree, st...
1    [yeah, expect, people, read, actually, accept,...
2    [although, realize, principle, strongest, poin...
3    [notwithstanding, legitimate, fuss, proposal, ...
4    [well, change, scoring, playoff, pool, unfortu...
Name: clean_doc, dtype: object

- 이제 각 단어에 정수 인코딩을 하는 동시에, 각 뉴스에서의 단어의 빈도수를 기록한다.
- 여기서는 각 단어를 `(word_id, word_frequency)`의 형태로 바꾸고자 한다.
  - `word_id` : 단어가 정수 인코딩된 값
  - `word_frequency` : 해당 뉴스에서의 해당 단어의 빈도수
- 이는 `gensim`의 `corpora.Dictionary()`를 사용하여 손쉽게 구할 수 있다.

- 전체 뉴스에 대해서 정수 인코딩을 수행하고, 두 번째 뉴스를 출력

In [14]:
from gensim import corpora

dictionary = corpora.Dictionary(tokenized_doc)
corpus = [dictionary.doc2bow(text) for text in tokenized_doc]
print(corpus[1]) # 수행된 결과에서 두번째 뉴스 출력. 첫번째 문서의 인덱스는 0

[(52, 1), (55, 1), (56, 1), (57, 1), (58, 1), (59, 1), (60, 1), (61, 1), (62, 1), (63, 1), (64, 1), (65, 1), (66, 1), (67, 1), (68, 1), (69, 2), (70, 1), (71, 1), (72, 1), (73, 1), (74, 1), (75, 2), (76, 1), (77, 1), (78, 1), (79, 1), (80, 1), (81, 1), (82, 1), (83, 1), (84, 1), (85, 1), (86, 2), (87, 1), (88, 1), (89, 1), (90, 1), (91, 1), (92, 1), (93, 1), (94, 2), (95, 1), (96, 1), (97, 1), (98, 1), (99, 1), (100, 1), (101, 1)]


- 위의 출력 결과 중에서 `(69, 2)`는 정수 인코딩이 66으로 할당된 단어가 두 번째 뉴스에서는 두 번 등장하였음을 의미한다.

<br>

- 69라는 값을 가지는 단어가 정수 인코딩이 되기 전에는 어떤 단어였는 지 확인
- 이는 `dictionary[]`에 기존 단어가 무엇인지 알고자 하는 정수값을 입력하여 확인할 수 있다.

In [15]:
print(dictionary[69])

faith


- 기존에는 단어 'faith' 이었음을 알 수 있다.

<br>

- 총 학습된 단어의 개수를 확인

In [16]:
len(dictionary)

70484

- 총 70,484개의 단어가 학습되었다.

<br>

### 2.5.2 LDA 모델 훈련시키기

- 기존의 뉴스 데이터가 총 20개의 카테고리를 가지고 있으므로 토픽의 개수를 20으로 하여 LDA 모델 학습

In [17]:
import gensim

NUM_TOPICS = 20

ldamodel = gensim.models.ldamodel.LdaModel(corpus,
                                           num_topics=NUM_TOPICS,
                                           id2word=dictionary,
                                           passes=15)
topics = ldamodel.print_topics(num_words=4)
for topic in topics:
    print(topic)

(0, '0.013*"turkish" + 0.010*"armenian" + 0.007*"armenians" + 0.007*"turks"')
(1, '0.007*"health" + 0.006*"public" + 0.006*"national" + 0.005*"information"')
(2, '0.034*"space" + 0.017*"nasa" + 0.007*"earth" + 0.007*"launch"')
(3, '0.022*"game" + 0.021*"team" + 0.016*"games" + 0.015*"play"')
(4, '0.014*"sale" + 0.012*"price" + 0.012*"shipping" + 0.011*"offer"')
(5, '0.009*"nrhj" + 0.006*"wwiz" + 0.005*"bxom" + 0.005*"gizw"')
(6, '0.011*"government" + 0.010*"people" + 0.009*"guns" + 0.008*"would"')
(7, '0.014*"like" + 0.009*"know" + 0.008*"would" + 0.006*"good"')
(8, '0.015*"president" + 0.010*"going" + 0.009*"stephanopoulos" + 0.009*"know"')
(9, '0.015*"bike" + 0.011*"engine" + 0.008*"ride" + 0.007*"cars"')
(10, '0.012*"would" + 0.009*"like" + 0.009*"good" + 0.008*"time"')
(11, '0.022*"israel" + 0.013*"israeli" + 0.008*"arab" + 0.008*"jews"')
(12, '0.011*"would" + 0.011*"people" + 0.007*"think" + 0.005*"know"')
(13, '0.014*"said" + 0.010*"people" + 0.009*"went" + 0.007*"armenians"')
(1

- 각 단어 앞에 붙은 수치는 단어의 해당 토픽에 대한 기여도를 보여준다.
- 또한 맨 앞에 있는 토픽 번호는 0부터 시작하므로 총 20개의 토픽은 0부터 19까지의 번호가 할당되어져 있다.

`passes`

- 알고리즘의 동작 횟루
- 알고리즘이 결정하는 토픽의 값이 적절히 수렴할 수 있도록 충분히 적당한 횟수를 정해주면 된다.
- 여기서는 총 15회를 수행했다.

`num_words=4`

- 총 4개의 단어만 출력하도록 함
- 만약 10개의 단어를 출력하고 싶다면 아래의 코드를 수행하면 된다.

In [20]:
for topic in ldamodel.print_topics():
    print(topic)

(0, '0.013*"turkish" + 0.010*"armenian" + 0.007*"armenians" + 0.007*"turks" + 0.006*"world" + 0.006*"jews" + 0.006*"turkey" + 0.006*"armenia" + 0.006*"russian" + 0.006*"char"')
(1, '0.007*"health" + 0.006*"public" + 0.006*"national" + 0.005*"information" + 0.005*"administration" + 0.005*"medical" + 0.005*"people" + 0.005*"states" + 0.004*"congress" + 0.004*"president"')
(2, '0.034*"space" + 0.017*"nasa" + 0.007*"earth" + 0.007*"launch" + 0.006*"satellite" + 0.006*"center" + 0.006*"research" + 0.006*"data" + 0.006*"orbit" + 0.005*"shuttle"')
(3, '0.022*"game" + 0.021*"team" + 0.016*"games" + 0.015*"play" + 0.013*"season" + 0.012*"hockey" + 0.009*"players" + 0.009*"league" + 0.009*"year" + 0.008*"period"')
(4, '0.014*"sale" + 0.012*"price" + 0.012*"shipping" + 0.011*"offer" + 0.009*"mail" + 0.009*"condition" + 0.009*"please" + 0.009*"asking" + 0.007*"sell" + 0.007*"best"')
(5, '0.009*"nrhj" + 0.006*"wwiz" + 0.005*"bxom" + 0.005*"gizw" + 0.004*"tbxn" + 0.004*"bhjn" + 0.004*"bxlt" + 0.004*

<br>

### 2.5.3 LDA 시각화 하기

- LDA 시각화를 위해서는 `pyLDAvis`의 설치가 필요하다.

In [21]:
!pip install pyLDAvis

     |████████████████████████████████| 1.6MB 2.8MB/s 
     |████████████████████████████████| 552kB 20.8MB/s 
  Created wheel for pyLDAvis: filename=pyLDAvis-2.1.2-py2.py3-none-any.whl size=97711 sha256=58bb10c442ef8c007b0522ccd497555463b08a4dc34ad77444a1c6cb69317ff9
  Stored in directory: /root/.cache/pip/wheels/98/71/24/513a99e58bb6b8465bae4d2d5e9dba8f0bef8179e3051ac414
  Created wheel for funcy: filename=funcy-1.14-py2.py3-none-any.whl size=32042 sha256=f8400cfe86a93fb30cd12b47d8aea375062385c3e71ec7b9b6b268965acb6bb3
  Stored in directory: /root/.cache/pip/wheels/20/5a/d8/1d875df03deae6f178dfdf70238cca33f948ef8a6f5209f2eb
Successfully built pyLDAvis funcy


```python
import pyLDAvis.gensim

pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(ldamodel, corpus, dictionary)
pyLDAvis.display(vis)
```

<img src="https://wikidocs.net/images/page/30708/visualization_final.PNG" />

**1) 좌측 그래프**

- 좌측의 원들은 각각의 20개의 토픽을 나타낸다.
- 각 원과의 거리는 각 토픽들이 서로 얼마나 다른 지를 보여준다.
- 만약 두 개의 원이 겹친다면, 이 두 개의 토픽은 유사한 토픽이라는 의미이다. 

**2) 우측 그래프**

- 위의 그림에서는 10번 토픽을 클릭하였고, 이에 따라 우측에는 10번 토픽에 대한 정보가 나타난다.
- 한 가지 주의할 점은 LDA 모델의 출력 결과에서는 토픽 번호가 0부터 할당되어 0 ~ 19의 숫자가 사용된 것과는 달리 위의 LDA 시각화에서는 토픽의 번호가 1부터 시작하므로 각 토픽의 번호는 1 ~ 20까지의 값을 가진다.

<br>

### 2.5.4 문서 별 토픽 분포 보기

- 위에서 토픽 별 단어 분포는 확있했다.
- 아직 문서 별 토픽 분포는 확인하지 못했다.

**문서 별 토픽 분포를 확인하는 방법**

- 각 문서의 토픽 분포는 이미 훈련된 LDA 모델인 `ldamodel[]`에 전체 데이터가 정수 인코딩된 결과를 넣은 후에 확인이 가능하다.

In [25]:
# 상위 5개의 문서에 대해서만 토픽 분포 확인
for i, topic_list in enumerate(ldamodel[corpus]):
    if i==5:
        break
    print(i, '번째 문서의 topic 비율은 ', topic_list)

0 번째 문서의 topic 비율은  [(0, 0.19782408), (6, 0.116727844), (10, 0.27996612), (11, 0.14733538), (12, 0.21991128), (13, 0.026945008)]
1 번째 문서의 topic 비율은  [(4, 0.2828452), (8, 0.0252698), (10, 0.17010671), (12, 0.50668395)]
2 번째 문서의 topic 비율은  [(6, 0.023512108), (9, 0.046444144), (11, 0.3035089), (12, 0.5822001), (16, 0.017814325)]
3 번째 문서의 topic 비율은  [(4, 0.01710697), (10, 0.49625656), (15, 0.021301458), (16, 0.4319461), (17, 0.022972245)]
4 번째 문서의 topic 비율은  [(3, 0.2731834), (7, 0.6178301), (17, 0.08065315)]


- 위의 출력 결과에서 `(숫자, 확률)`은 각각 토픽 번호와 해당 토픽이 해당 문서에서 차지하는 분포도를 의미한다.
- ex) 0 번째 문서의 토픽 비율 중 `(0, 0.19782408)`
  - 0 번째 토픽이 19%의 분포도를 가지는 것을 의미한다.

- 위의 코드를 응용하여 좀 더 깔끔한 형태인 데이터프레임 형식으로 출력

In [ ]:
def make_topictable_per_doc(ldamodel, corpus,texts):
    topic_table = pd.DataFrame()

    # 몇 번째 문서인지를 의미하는 문서 번호와 해당 문서의 토픽 비중을 한 줄씩 꺼냄
    for i, topic_list in enumerate(ldamodel[corpus]):
        doc = topic_list[0] if ldamodel.per_word_topics else topic_list
        # 각 문서에 대해서 비중이 높은 토픽순으로 토픽을 정렬
        doc = sorted(doc, key=lambda x: (x[1]), reverse=True)
        # EX) 정렬 전 0번 문서 : (2번 토픽, 48.5%), (8번 토픽, 25%), (10번 토픽, 5%), (12번 토픽, 21.5%)
        # Ex) 정렬 후 0번 문서 : (2번 토픽, 48.5%), (8번 토픽, 25%), (12번 토픽, 21.5%), (10번 토픽, 5%)
        # --> 48 > 25 > 21 > 5 순으로 정렬이 된 것.

        # 모든 문서에 대해서 각각 아래를 수행
        for j, (topic_num, prop_topic) in enumerate(doc): # 몇 번 토픽인지와 비중을 나눠서 저장
            if j == 0: # 정렬을 한 상태이므로 가장 앞에 있는 것이 가장 비중이 높은 토픽
                topic_table = topic_table.append(pd.Series([int(topic_num), # 가장 비중이 높은 토픽
                                                            round(prop_topic,4), # 가장 비중이 높은 토픽의 비중
                                                            topic_list]), # 전체 토픽의 비중
                                                 ignore_index=True)
            else:
                break
    
    return (topic_table)

In [29]:
topictable = make_topictable_per_doc(ldamodel, corpus, tokenized_doc)

# 문서 번호를 의미하는 열(column)로 사용하기 위해 인덱스 열을 하나 더 만든다.
topictable = topictable.reset_index()

topictable.columns = ['문서 번호', '가장 비중이 높은 토픽', '가장 높은 토픽의 비중', '각 토픽의 비중']
topictable[:10]

,문서 번호,가장 비중이 높은 토픽,가장 높은 토픽의 비중,각 토픽의 비중
0,0,10.0,0.2799,"[(0, 0.1978215), (6, 0.116709985), (10, 0.2799..."
1,1,12.0,0.5067,"[(4, 0.2828397), (8, 0.02526897), (10, 0.17011..."
2,2,12.0,0.5822,"[(6, 0.023517087), (9, 0.04644332), (11, 0.303..."
3,3,10.0,0.4963,"[(4, 0.017109115), (10, 0.4962679), (15, 0.021..."
4,4,7.0,0.6178,"[(3, 0.27318653), (7, 0.6178265), (17, 0.08065..."
5,5,12.0,0.6173,"[(4, 0.025973642), (7, 0.30248526), (12, 0.617..."
6,6,5.0,0.5942,"[(4, 0.019458063), (5, 0.59416246), (10, 0.186..."
7,7,12.0,0.5100,"[(3, 0.13856046), (7, 0.19688572), (11, 0.1420..."
8,8,7.0,0.5565,"[(6, 0.30395514), (7, 0.5565154), (8, 0.077192..."
9,9,10.0,0.3437,"[(1, 0.029589685), (4, 0.03796072), (7, 0.0812..."
